In [6]:
pip install numpy pandas tensorflow matplotlib nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.3 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 2.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

In [12]:
text_df = pd.read_csv("../data/putin.csv")
text = text_df.values

def testmap(a):
  return a[0]

joined_text = " ".join(map(testmap, text))

with open("../data/joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [13]:
partial_text = joined_text[:1000000]

In [14]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [15]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [16]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [17]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [18]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [19]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

2023-04-20 05:07:26.463406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 05:07:26.465761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 05:07:26.466897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [20]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10


2023-04-20 05:07:50.288332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 05:07:50.290591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 05:07:50.292336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

690/690 [==============================] - 329s 473ms/step - loss: 8.1554 - accuracy: 0.0459
Epoch 2/10
690/690 [==============================] - 382s 554ms/step - loss: 8.0476 - accuracy: 0.0497
Epoch 3/10
690/690 [==============================] - 315s 456ms/step - loss: 8.5042 - accuracy: 0.0546
Epoch 4/10
690/690 [==============================] - 312s 452ms/step - loss: 8.7663 - accuracy: 0.0629
Epoch 5/10
690/690 [==============================] - 313s 453ms/step - loss: 8.5614 - accuracy: 0.0766
Epoch 6/10
690/690 [==============================] - 309s 448ms/step - loss: 8.1896 - accuracy: 0.0937
Epoch 7/10
690/690 [==============================] - 318s 462ms/step - loss: 7.7844 - accuracy: 0.1132
Epoch 8/10
690/690 [==============================] - 312s 452ms/step - loss: 7.3137 - accuracy: 0.1435
Epoch 9/10
690/690 [==============================] - 311s 451ms/step - loss: 6.7244 - accuracy: 0.1834
Epoch 10/10
690/690 [==============================] - 308s 446ms/step - lo

In [19]:
history = model.fit(X, y, batch_size=128, epochs=5, shuffle=True).history

Epoch 1/5
1326/1326 [==============================] - 421s 316ms/step - loss: 5.5365 - accuracy: 0.2450
Epoch 2/5
1326/1326 [==============================] - 428s 323ms/step - loss: 5.2118 - accuracy: 0.2751
Epoch 3/5
1326/1326 [==============================] - 426s 321ms/step - loss: 4.9456 - accuracy: 0.3037
Epoch 4/5
1326/1326 [==============================] - 427s 322ms/step - loss: 4.6771 - accuracy: 0.3346
Epoch 5/5
1326/1326 [==============================] - 426s 322ms/step - loss: 4.5058 - accuracy: 0.3530


In [21]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [21]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [22]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [23]:
possible = predict_next_word("Дорогие друзья", 5)

2023-04-20 06:02:15.611968: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 06:02:15.613698: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 06:02:15.615018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 865ms/step


In [24]:
for idx in possible:
    print(unique_tokens[idx])

и
в
да
со
вместе


In [25]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [26]:
generate_text("Дорогие товарищи, я", 100, 10)

1/1 [==============================] - 0s 21ms/step


'Дорогие товарищи, я мы решение да решения мы со в газа решения им при государство борьба они сохранить развития не со сейчас которую вместе со дорогие ещё вместе вместе против со весь вместе и её да развития кроме при да политические в для до тоже да от сейчас ещё при ещё и поэтому вместе поэтому им вместе им наши это при при более под всё мы и без со мы политические человека это которые или вместе решения при всегда мировые от на до одновременно решение сегодня здесь наших ещё хочу будет от со дорогие им на при политические жизни с до в конечно'

In [34]:
generate_text("Специальная военная операция", 100, 5)

1/1 [==============================] - 0s 22ms/step


'Специальная военная операция последствия своим весь нато также для со донбасса особо при весь более им человека им со со практически тоже тоже для в в при в необходимо более более людей все их да со ещё люди решение мы да в и их при со от со против более человека людей им их вместе со для вместе донбасса вместе особо в необходимо да при со сейчас им при со со более решение вместе система благодаря вместе вместе их в наши здесь со ещё вместе ещё и хочу ещё ещё ещё и на и в в в при и более против против со'

In [39]:
for idx in predict_next_word("The president will most likely not be there to help", 5):
    print(unique_tokens[idx])


american
the
our
us
president
